# Import Statements

In [1]:
import gurobipy as gp
import networkx as nx
from networkx.algorithms.approximation.steinertree import steiner_tree as steiner_tree_2
import random
from math import log
import scipy
from scipy.optimize import newton
from scipy.optimize import root_scalar
import sympy

# Constants

## Debug Level Flags

In [2]:
DEBUG_LEVEL_NONE = 0
DEBUG_LEVEL_THEORY_1 = 1.1
DEBUG_LEVEL_THEORY_2 = 1.2
DEBUG_LEVEL_FULL = 2
DEBUG_LEVEL_EXTREME = 3

## General Constants

In [3]:
DEBUG_LEVEL = DEBUG_LEVEL_THEORY_1
NUM_NODES = 100
NUM_EDGES = 150
DEGREE = 3
NUM_MULTICAST_REQUESTS = 50
MAX_MULTICAST_SIZE = NUM_NODES//5
TOLERANCE = 0.1

NATURAL = 0
UNIFORM = 1
PERTURB = 2

# Generating Problem Instances

In [4]:
def is_connected(G):
    return nx.is_k_edge_connected(G, 1)

def get_random_connected_graph():
    G = nx.empty_graph(NUM_NODES)
    while not is_connected(G):
        G = nx.random_regular_graph(DEGREE, NUM_NODES)
    return G

class MulticastRequest:
    def __init__(self, size, graph, source=None, recipients=None):
        if source is None and recipients is None:
            multicast_group = random.sample(list(graph), size)
            source = multicast_group[0]
            recipients = set(multicast_group[1:])
        self.source = source
        self.recipients = recipients
        self.size = len(recipients) + 1
    
    def multicast_group(self):
        return self.recipients.union([self.source])

class MulticastPackingInstance:
    def __init__(self, num_requests, max_request_size, graph=None, requests=None):
        if graph is None:
            graph = get_random_connected_graph()
        if requests is None:
            requests = list()
            for i in range(num_requests):
                k = random.randint(2, max_request_size)
                requests.append(MulticastRequest(k, graph))
        self.graph = graph
        self.num_edges = len(self.graph.edges())
        self.requests = requests
        self.num_requests = len(requests)

# Jansen-Zhang General Convex min-max

In [5]:
def theta_eq(theta, x, t, M, f):
    retval = 1
    for e in self.instance.graph.edges():
        retval = retval - t*theta/(M*(theta - f(x,e)))
    print("theta = {}".format(theta))
    print("theta_eq = {}".format(retval))
    return retval

def derivative_theta_eq(theta, x, t, M, f):
    retval = 0
    for e in self.instance.graph.edges():
        retval = retval + t*f(x,e)/(M*(theta - f(x,e))*(theta - f(x,e)))
    print("d_theta_eq = {}".format(retval))
    return retval

class JansenZhangMinMaxer:
    sigma0 = 1
    
    def __init__(self, constraint_indices, f, approx_block_solver):
        self.constraint_indices = constraint_indices
        self.M = len(constraint_indices)
        self.f = f
        self.abs = approx_block_solver
        self.current_solution = None
        self.lamb = dict()
        self.t = self.sigma0/6
        self.prices = dict()
        self.theta = dict()
    
    # Define a function that computes theta_t(x)
    def theta_via_sympy(self, x, t):
        M = self.M
        f = self.f
        
        theta = sympy.Symbol('theta')
        expression = 1
        for m in constraint_indices:
            expression -= (t/M) * theta/(theta - f(x,m))
        soln_set = sympy.solvers.solveset(expression, theta, domain=sympy.Interval(self.lamb[x], sympy.S.Infinity, True, True))
        #assert len(soln_set == 1)
        if DEBUG_LEVEL >= DEBUG_LEVEL_FULL:
            print(soln_set)
        for soln in soln_set:
            retval = soln
        return retval
        
    def calculate_theta(x, t):
        return scipy.optimize.brentq(theta_eq, self.lamb[x]/(1-t/self.M), self.lamb[x]/(1-t), args=(x, t, self.M, self.f))
        #return scipy.optimize.newton(theta_eq, x0, derivative_theta_eq, args=(x, t, self.M, self.f))
    
    def get_theta(x, t):
        if (x,t) not in self.theta:
            self.theta[(x,t)] = self.calculate_theta(x,t)
        return self.theta[(x,t)]
    
    # Define a function that computes p_t_e(x)
    def get_price(m,x,t):
        M = self.M
        f = self.f
        
        if (x,t) not in prices:
            prices[(x,t)] = [None]*M
            for n in constraint_indices:
                prices[(x,t)][n] = t*get_theta(x,t)/(M*(get_theta(x,t) - f(x,n)))
        return prices[(x,t)][m]

# Create (Reduced) LP

In [6]:
def create_LP(G, multicast_requests):
    nx.set_edge_attributes(G, 1, "weight")
    reduced_LP = gp.Model("Multicast Packing Model - Reduced")
    congestion = reduced_LP.addVar(name="lambda")
    reduced_LP.setObjective(congestion, gp.GRB.MINIMIZE)
    reduced_LP.update()
    # Packing Constraints
    for e in G.edges():
        reduced_LP.addConstr(congestion >= 0, name="{} congestion".format(sorted(e)))
    # Simplex Constraints
    for i in range(len(multicast_requests)):
        reduced_LP.addConstr(0*congestion == 1, name="Tree Selection for {}".format(i))
    reduced_LP.update()
    return reduced_LP

# Column Generating Subproblem

In [7]:
def cost(G):
        assert nx.is_weighted(G)
        return sum (nx.get_edge_attributes(G, "weight").values())

class MulticastPackingColumnGenerator:
    def __init__(self, instance, reduced_LP):
        self.instance = instance
        self.reduced_LP = reduced_LP
        self.generate_tree = lambda i: nx.algorithms.approximation.steinertree.steiner_tree(self.instance.graph, self.instance.requests[i].multicast_group())
        
    def generate_new_trees(self):
        new_trees = list()
        assert nx.is_weighted(self.instance.graph)
        for i in range(self.instance.num_requests):
            new_tree = self.generate_tree(i)
            new_trees.append(new_tree)
            
            coeffs = list()
            constrs = list()
            # the new tree contributes to the constraint for each of its edges
            for e in new_tree.edges():
                coeffs.append(-1)
                constrs.append(self.reduced_LP.getConstrByName("{} congestion".format(sorted(e))))
            
            # the new tree contributes to the selection constraint for the corresponding multicast request
            coeffs.append(1)
            constrs.append(self.reduced_LP.getConstrByName("Tree Selection for {}".format(i)))
            self.reduced_LP.addVar(name="x{}{}".format(i, nx.info(new_tree)), column=gp.Column(coeffs, constrs))
            
        self.reduced_LP.update()
        return new_trees

# Main Solver

In [8]:
class MulticastPackingSolver:
    
    def get_next_solution_via_column_gen(self):
        self.reduced_LP.optimize()
        return self.reduced_LP.getVars()

    def __init__(self, instance=None):
        if instance is None:
            instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
        self.instance = instance
        self.reduced_LP = create_LP(self.instance.graph, self.instance.requests)
        self.column_generator = MulticastPackingColumnGenerator(self.instance, self.reduced_LP)
        self.tol = TOLERANCE
        self.stop_flag = False
        self.price_flag = NATURAL
        self.iteration = 0
        self.solution = list()
        self.lamb = dict()
        self.price = dict()
        self.uniform_price = dict()
        self.multicast_costs = dict()
        
        # The main solver will have a copy of the general solver in Jansen-Zhang for ease of comparison
        def f(x,e):
            return self.lamb[x] + reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Slack) 
            # Note: Gurobi signs their slacks stupidly
            
        def approx_block_solver(price, t):
            nx.set_edge_attributes(self.instance.graph, price, "weight")
            new_trees = self.column_generator.generate_new_trees()
            new_vars = list()
            for tree in new_trees:
                new_vars.append(self.reduced_LP.getVarByName("x{}{}".format(len(new_vars), nx.info(tree))))
        
            
        self.jz_minmaxer = JansenZhangMinMaxer(self.instance.graph.edges(), f, approx_block_solver)
        
        self.get_next_solution = self.get_next_solution_via_column_gen

    
    def perform_iteration(self):
        new_trees = self.column_generator.generate_new_trees()
        x = tuple(self.get_next_solution())
        self.solution.append(x)
        
        if DEBUG_LEVEL >= DEBUG_LEVEL_EXTREME:
            print(reduced_LP.getA().toarray())
            print(reduced_LP.getAttr("RHS"))
        
        self.lamb[x] = self.reduced_LP.getObjective().getValue()
        
        if x not in self.price:
            self.price[x] = dict()
            self.uniform_price[x] = dict()
        
        for e in self.instance.graph.edges():
            self.price[x][e] = self.reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Pi)
            if self.price[x][e] > 0:
                self.uniform_price[x][e] = 1
            else:
                self.uniform_price[x][e] = 0
            
        if self.price_flag == NATURAL:
            nx.set_edge_attributes(self.instance.graph, self.price[x], "weight")
        if self.price_flag == UNIFORM:
            nx.set_edge_attributes(self.instance.graph, self.uniform_price[x], "weight")
        if self.price_flag == PERTURB:
            nx.set_edge_attributes(self.instance.graph, lambda e : self.jz_minmaxer.get_prices(e, x, self.tol), "weight")
        
        # Some print statements for comparing different pricing strategies. Needs to be reworked.
        if False:
            new_trees_uniform = generate_new_trees(G, multicast_requests, uniform_prices)
            new_trees_perturbed = generate_new_trees(G, multicast_requests, perturbed_prices)
            newVsUniform = [not any(new_trees[i].edges() - new_trees_uniform[i].edges()) for i in range(len(multicast_requests))]
            newVsPerturbed = [not any(new_trees[i].edges() - new_trees_perturbed[i].edges()) for i in range(len(multicast_requests))]
            UniformVsPerturbed = [not any(new_trees_uniform[i].edges() - new_trees_perturbed[i].edges()) for i in range(len(multicast_requests))]
            print("New Tree = New Trees with Uniform Prices: {}".format(all(newVsUniform)))
            print("New Tree = New Trees with Uniform Prices: {}".format(all(newVsPerturbed)))
            print("New Tree = New Trees with Uniform Prices: {}".format(all(UniformVsPerturbed)))
        
        if x not in self.multicast_costs:
            self.multicast_costs[x] = [None] * len(self.instance.requests)
        for i in range(self.instance.num_requests):
            self.multicast_costs[x][i] = self.reduced_LP.getConstrByName("Tree Selection for {}".format(i)).getAttr(gp.GRB.Attr.Pi)
        
        if all([cost(new_trees[i]) - self.multicast_costs[x][i] >= 0 for i in range(self.instance.num_requests)]):
            stop_flag = True
        if sum([cost(new_trees[i]) for i in range(self.instance.num_requests)]) >= self.lamb[x]:
            stop_flag = True
        
        if DEBUG_LEVEL >= DEBUG_LEVEL_THEORY_1:
            print(self.iteration)
            print("lambda(x): {}".format(self.lamb[x]))
            print("log lambda(x): {}".format(log(self.lamb[x])))
            
        if DEBUG_LEVEL >= DEBUG_LEVEL_THEORY_2:
            for e in self.instance.graph.edges():
                if prices[x][e] > 0:
                    t = sympy.Symbol('t')
                    p = sympy.limit_seq(jz_price(x, t, f, G, lambda_of_x), t)
                    p = jz_price(x, e, TOLERANCE, f, G, lambda_of_x)
                    print("{}: price: {} \t JZ price: {} \t slack: {}".format(e, prices[e], p, reduced_LP.getConstrByName("{} congestion".format(sorted(e))).getAttr(gp.GRB.Attr.Slack)))
            
        self.iteration += 1

# Testing

In [9]:
def test_instance_generation(num_tests):
    assert not is_connected(nx.empty_graph(NUM_NODES))
    assert is_connected(get_random_connected_graph())
    G = get_random_connected_graph()
    for i in range(num_tests):
        request = MulticastRequest(MAX_MULTICAST_SIZE, G)
        source = request.source
        recipients = request.recipients
        assert request.source is not None
        assert request.recipients
        assert not request.source in request.recipients
        assert isinstance(request.recipients, set)
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    for request in instance.requests:
        assert request.source in instance.graph
        assert request.recipients.issubset(instance.graph)

def test_LP_creation(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    assert len(reduced_LP.getVars()) == 1
    assert len(reduced_LP.getConstrs()) == NUM_EDGES + NUM_MULTICAST_REQUESTS, "Num of constraints: {} Expected: {}".format(len(reduced_LP.getConstrs()), NUM_EDGES + NUM_MULTICAST_REQUESTS)
    
def test_subproblem(num_tests):
    instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
    reduced_LP = create_LP(instance.graph, instance.requests)
    column_generator = MulticastPackingColumnGenerator(instance, reduced_LP)
    
    new_trees = column_generator.generate_new_trees()
    assert (len(new_trees) == instance.num_requests)
    for tree in new_trees:
        assert(cost(tree) == len(tree.edges()))
           
    assert(len(reduced_LP.getVars()) == 1+instance.num_requests)
    
def run_tests(num_tests):
    test_instance_generation(num_tests)
    test_LP_creation(num_tests)
    test_subproblem(num_tests)

NUM_TESTS = 100
run_tests(NUM_TESTS)

# Run the solver

In [ ]:
for i in range(1):
    print("BEGIN NEW TEST")
    solver = MulticastPackingSolver()
    while(not solver.stop_flag):
        solver.perform_iteration()
    
    #print("Natural Prices")
    #solve_multicast_packing_via_columngen(*instance, priceFlag=NATURAL)
    #print("Uniform Prices")
    #solve_multicast_packing_via_columngen(*instance, priceFlag=UNIFORM)
    #print("Perturbed Prices")
    #solve_multicast_packing_via_columngen(*instance, priceFlag=PERTURB)

BEGIN NEW TEST
0
lambda(x): 16.0
log lambda(x): 2.772588722239781
1
lambda(x): 14.0
log lambda(x): 2.6390573296152584
2
lambda(x): 12.823529411764707
log lambda(x): 2.551281718732873
3
lambda(x): 11.593642000406586
log lambda(x): 2.4504568444355086
4
lambda(x): 10.911700929753922
log lambda(x): 2.389835693261615
5
lambda(x): 10.535109452609953
log lambda(x): 2.354713436628851
6
lambda(x): 10.348407506450227
log lambda(x): 2.3368326437619924
7
lambda(x): 10.142546862810336
log lambda(x): 2.316739136527558
8
lambda(x): 10.061903928175655
log lambda(x): 2.3087564040386166
9
lambda(x): 9.95819540510796
log lambda(x): 2.2983958709545536
10
lambda(x): 9.88230442644788
log lambda(x): 2.290745726106504
11
lambda(x): 9.759250718820672
log lambda(x): 2.2782156268653506
12
lambda(x): 9.685939834093054
log lambda(x): 2.270675332325678
13
lambda(x): 9.572367114099631
log lambda(x): 2.2588805222253856
14
lambda(x): 9.495966334257975
log lambda(x): 2.2508671120460253
15
lambda(x): 9.444454169122672
l